In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

base_path = "/content/drive/MyDrive/Colab Notebooks/platform_subscribe/"

train = pd.read_csv(base_path + "train.csv")
test = pd.read_csv(base_path + "test.csv")
submission = pd.read_csv(base_path + "sample_submission.csv")

In [3]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.6 MB/s eta 0:00:00


In [4]:
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 평가 지표
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

evulation_metric = f1_score

### second_best = ['average_time_per_learning_session', 'recent_learning_achievement'] + data_scaling + catboost

In [5]:
second_best = ['average_time_per_learning_session', 'recent_learning_achievement']
third_best = ['average_time_per_learning_session',
   'recent_learning_achievement',
   'average_login_time',
   'subscription_duration',
   'recent_login_time',
   'monthly_active_learning_days',
   'total_completed_courses',
   'abandoned_learning_sessions',
   'payment_pattern',
   'customer_inquiry_history']

In [6]:
X=train[third_best]
y=train.target
X

,average_time_per_learning_session,recent_learning_achievement,average_login_time,subscription_duration,recent_login_time,monthly_active_learning_days,total_completed_courses,abandoned_learning_sessions,payment_pattern,customer_inquiry_history
0,8.427187,68.360455,14.946163,13,14,18,16,3,5,4
1,72.646087,97.567322,18.453224,16,18,16,13,2,6,1
2,21.774492,94.358763,16.195228,22,1,13,14,3,7,0
3,42.659066,70.153228,17.628656,1,19,19,18,0,0,1
4,30.744287,81.917908,21.390656,4,5,19,10,2,0,3
...,...,...,...,...,...,...,...,...,...,...
9995,84.053558,64.966803,14.727623,22,29,18,16,2,1,1
9996,45.464833,82.750244,19.374054,10,11,9,8,3,7,2
9997,127.302411,81.567839,18.240978,7,27,24,14,3,6,1
9998,5.297234,89.885656,18.783800,11,7,10,10,4,0,2


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.25, random_state=42)

In [8]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
std.fit(X_train)
X_train_scaled = std.transform(X_train)
X_val_scaled = std.transform(X_val)

In [9]:
clf_cb = CatBoostClassifier()
clf_cb.fit(X_train_scaled, y_train)
pred_train_cb=clf_cb.predict(X_train_scaled)
pred_val_cb=clf_cb.predict(X_val_scaled)

evulation_metric_train_cb=evulation_metric(y_train, pred_train_cb)
evulation_metric_val_cb=evulation_metric(y_val, pred_val_cb)
print(evulation_metric_train_cb)
print(evulation_metric_val_cb)

Learning rate set to 0.022141
0:	learn: 0.6909054	total: 52.5ms	remaining: 52.5s
1:	learn: 0.6882101	total: 59.5ms	remaining: 29.7s
2:	learn: 0.6858272	total: 67.8ms	remaining: 22.5s
3:	learn: 0.6841863	total: 76.8ms	remaining: 19.1s
4:	learn: 0.6817008	total: 83.9ms	remaining: 16.7s
5:	learn: 0.6794617	total: 92.8ms	remaining: 15.4s
6:	learn: 0.6780223	total: 112ms	remaining: 15.9s
7:	learn: 0.6759043	total: 119ms	remaining: 14.7s
8:	learn: 0.6738095	total: 127ms	remaining: 14s
9:	learn: 0.6720928	total: 138ms	remaining: 13.6s
10:	learn: 0.6702767	total: 143ms	remaining: 12.8s
11:	learn: 0.6693454	total: 152ms	remaining: 12.5s
12:	learn: 0.6684868	total: 160ms	remaining: 12.2s
13:	learn: 0.6670193	total: 168ms	remaining: 11.8s
14:	learn: 0.6656340	total: 177ms	remaining: 11.6s
15:	learn: 0.6648775	total: 181ms	remaining: 11.1s
16:	learn: 0.6637017	total: 185ms	remaining: 10.7s
17:	learn: 0.6625718	total: 188ms	remaining: 10.3s
18:	learn: 0.6618136	total: 202ms	remaining: 10.4s
19:	lea

### test 확인

In [10]:
X_test_scaled = std.transform(X_test)
X_test_scaled

array([[-0.73077477,  0.90464221,  0.2408262 , ..., -1.73968925,
         0.63310885,  0.69036171],
       [-0.77659011, -0.61644965, -0.38008139, ..., -0.01304575,
         0.2031051 , -0.01134425],
       [-0.51640995,  1.40444115, -0.26540216, ..., -0.58859358,
        -0.65690239,  0.69036171],
       ...,
       [-0.81444743, -0.32212759, -0.5779501 , ..., -0.58859358,
        -1.51690989,  0.69036171],
       [-0.49334975, -0.03994693,  2.04490767, ..., -0.01304575,
         0.2031051 ,  0.69036171],
       [-0.13814892,  0.41189355, -0.10698085, ..., -0.01304575,
        -0.65690239, -0.7130502 ]])

In [ ]:
pred_test_cb=clf_cb.predict(X_test_scaled)

evulation_metric_test_cb=evulation_metric(y_test, pred_test_cb)
print(evulation_metric_test_cb)

0.7472668810289389
